# Module 4, Week 1 In Class Exercise

Non-linear model fitting

**Before class reading: Stein and Stein, 1992; Fundamentals of Geophysics Chapter 4.2 **

**Last week we:**
- Downloaded Anaconda to your local machine
- Launched a fresh Jupyter Notebook
- Launched Spyder
- Wrote a program to fit the orbital period and distance of planets to prove Kepler's 3rd law

**Our goals for today:**
- Load marine bathymetry data and age of seafloor (found from marine magnetics) from several oceanic ridges
- Use `interpolate.Rbf` to do a 2D interpolation of age data
- Plot bathymetry data vs. age
- Fit a bathymetry(age) model
- Compare our with the model of Stein and Stein, 1992



## Setup

Run this cell as it is to setup your environment.

In [ ]:
import  numpy  as  np
import  matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from scipy import interpolate
from cartopy import config
import cartopy.crs as ccrs

##  Part 1: Marine bathymetry and Age, Theory

Earth is constantly losing heat from its interior. This internal heat has two main sources 1) the formation of Earth and 2) heat produced by the decay of radioactive isotopes. Heat can be transmitted 3 ways 1) conduction 2) convection and 3) radiation. In the system we'll look at today - the flow of heat out of the surface of the earth - the relavent transport mechanism is conduction. The transport of heat by conduction in a solid is governed by a simple equation. Consider a solid bar of length $L$ and cross-sectional area $A$ with its ends maintained at temperatures $T_{1}$ and $T_{2}$, respectively (Fig.4.15a). Assuming that heat flows only along the bar (i.e., there are no side losses) the net amount of heat ($\Delta  Q$) that passes in a given time from the hot end to the cold end depends directly on the temperature difference ($T_{2}-T_{1}$), the area of cross-section ($A$) and the time of observation ($\Delta  t$), and inversely on the length of the bar ($L$). These observations can be summarized in the equation

$\Delta Q =kA\frac{T_{2}-T_{1}}{L}\Delta t$

<img src="Figures/simple_model.png" width=400>

> Source: Fundamentals of Geophysics (2nd Edition) Lowrie, W.

The constant of proportionality, $k$, is the thermal conductivity, which is a property of the material of the bar. If
the length of the bar is very small or the temperature change across it is uniform, the ratio $(T_2-T_1)/L$ is the temperature gradient. We can modify the equation to describe the vertical flow of heat out of the Earth by substituting the vertical temperature gradient, ($\frac{\mathrm{d} T}{\mathrm{d} z}$), which is also called the geothermal gradient. The aboave equation can then be rearranged as follows

$q_{z} = \frac{1}{A}\frac{dQ}{dt}= k\frac{dT}{dz}$

In this equation $q_{z}$ is the heat flux, defined as the flow of heat per unit area per second. If temperature increases in the downward (negative) direction of the z-axis, the flow of heat from high to low temperature is upward (postive) (Fig.4.15b).

<img src="Figures/1D_heat_flux_model.png" width=400>

> Source: Fundamentals of Geophysics (2nd Edition) Lowrie, W.

Consider the flow of heat in the positive direction of the z-axis through a small rectangular prism with sides $\mathrm{d}x$, $\mathrm{d}y$ and $\mathrm{d}z$ (see Fig. 4.15b). We will assume that there are no sources of heat inside the box. Let the amount of heat entering the prism at ($z \cdot \mathrm{d}z$) be $Q_{z}$. This is equal to the heat flow $q_{z}$ multiplied by the area of the surface it flows across ($\mathrm{d}x \mathrm{d}y$) and by the duration of the flow ($\mathrm{d}t$). The heat leaving the box at $z$ is $Q_{z}-\mathrm{d}Q_{z}$, which can be written $Q_{z}-(\mathrm{d}Q_{z} /\mathrm{d}z)\mathrm{d}z$. The increase in heat in the small box is the difference between these amounts:

$\frac{\mathrm{d} Q_{z}}{\mathrm{d} z}\mathrm{d} z = \frac{\mathrm{d} q_{z}}{\mathrm{d} z}\mathrm{d} z(\mathrm{d} x \mathrm{d} y)\mathrm{d} t=k\frac{\mathrm{d}^2 T}{\mathrm{d} z^{2}}\mathrm{d} V\mathrm{d} t$

where $\mathrm{d} V$ is the volume of the box ($\mathrm{d} x \mathrm{d} y \mathrm{d} z$). The heat increase in the
box causes its temperature to rise by an amount ($\mathrm{d} T$), determined by the specific heat at constant pressure ($c_{p}$) and the mass of material ($m$) in the box:

$c_{p}m\mathrm{d} T=c_{p}\rho  \mathrm{d} V \mathrm{d} T$

where $\rho$ is the density of the material in the box. The equation for conduction is then:

$\frac{\partial T}{\partial t}= \kappa \frac{\partial^2 T}{\partial z^2}$.

The the solution to this equation (i.e. 1D cooling of a semi-infinite half space) is:

$T(z,t) = T_{m} \mathrm{erf}(\frac{z}{2 \sqrt{\kappa t}})$.

The heat loss from the cooling lithosphere can be computed from this temperature distribution. The heat flow is proportional to the temperature gradient. The heat flowing out of a semi-infinite half-space is thus obtained by differentiating the above equation for $T(z,t)$ with respect to $z$:

$\begin{matrix}
q_{z} & = k\frac{\partial T}{\partial z} =\frac{\partial \eta }{\partial z}\frac{\partial T }{\partial  \eta} \\ 
 &  = k\frac{1}{2 \sqrt{\kappa t}}\frac{\partial T_{m} \mathrm{erf}(\eta)}{\partial \eta}  \\
 & = \frac{k T_{m}}{2 \sqrt{\kappa t}}\frac{\partial }{\partial \eta} \frac{2}{\sqrt{\pi}}\left \{ \int_{0}^{\eta} e^{-u^{2}}\mathrm{d}u\right \}
\end{matrix}$

which simplifies to 


$q_{z}= \frac{k T_{m}}{2 \sqrt{\kappa t}} e^{-\eta^{2}}$

At the surface, $z=0$, $\eta=0$, and exp($-\eta^2$)=1. The surface heat flow at time $t$ is

$q_{z} = \frac{kT_{m}}{\sqrt{\pi \kappa t}}$

The semi-infinite half-space is quite a good model for the cooling of oceanic lithosphere. The oceanic heat flow indeed varies with distance from an oceanic ridge as $1/\sqrt{t}$, where $t$ is the corresponding age of the lithosphere.

<img src="Figures/global_heat_flow.png" width=700>

> Source: Fundamentals of Geophysics (2nd Edition) Lowrie, W.

The heat flow is highest near to the ridge and decreases with increasing distane from it. Older lithosphere cools slightly less repidly, so an exponential decay model works better. The decrease in heat flow with age is explained by a global model called the Global Depth and Heat Flow model (GDH1). The model predicts the following relationships between heat flow ($q$, mW m$^{-2}$) and age ($t$, Ma): 

$q(t) =k\frac{T_{m}}{\sqrt{\pi \kappa t}} = 510 t^{-1/2} \quad \quad (t\leqslant 55 Ma)$

$q(t) = q_{s}[1 + 2 \mathrm{exp}(-\kappa \pi^{2} t / a^{2})] = 48 + 96 \mathrm{exp}(-0.0278t)\quad \quad (t > 55 Ma)$

Here $q_{s}$ is the asymptotic heat flow, to which the heat flow decreases over very old oceanic crust ($ \approx$ 48 mW m$^{-2}$), $a$ is the asymptotic thickness of old oceanic lithosphere ($ \approx$ 95 km), and $\kappa$ is its thermal diffusivity ($\approx 0.8 \times 10^{–6}$ m$^{2}$ s$^{-1}$).

<img src="Figures/depth_age_setup.png" width=500>

> Source: Fundamentals of Geophysics (2nd Edition) Lowrie, W.

The ridge system is isostatically compensated. As hot material injected at the ridge crest cools, its volume contracts and its density increases. To maintain isostatic equilibrium a vertical column sinks into the supporting substratum as it cools. Consequently, the depth of the ocean floor (the top surface of the column) is expected to increase with age of the lithosphere. The cooling half-space model predicts an increase in depth proportional to , where t is the age of the lithosphere, and this is observed up to an age of about 20 Ma. Beyond 20 Ma the data are better fitted by an exponential decay. The optimum relationships between bathymetry ($b$, m) and age ($t$, Ma) can be written

$b(t) = b_{r} + w = b_{r} + \frac{2 \alpha \rho_{m}T_{m} \sqrt{\kappa}}{\sqrt{\pi} (\rho_{m}-\rho_{w})} \sqrt{t} \quad \quad (t< 20 Ma)$

$b(t) = b_{r} + b_{s}[1 - (8/\pi^{2}) \mathrm{exp}(-\kappa \pi^{2} t / a^{2})]\quad \quad (t \geqslant 20 Ma)$

where $b_{r}$ is the mean bathymetric-depth of the ocean floor at ridge crests, $b_s$ is the asymptotic subsidence of old lithosphere and the other parameters are as before.

These notes are paraphrased and abridged from Fundamentals of Geophysics (2nd Edition) Lowrie, W.

<img src="Figures/Stein_1992_fig1.png" width=500>

> Source: Stein, C. and Stein S. (1992). _A model for the global variation in oceanic depth and heat flow with lithospheric age_. Nature, 359, 123-129.

GDH1 model for bathymetry:

$b(t) = -2600 - 365 \sqrt{t} \quad  \quad        t < 20$ Myr

$b(t) = -5651 + 2473 e^{-0.0278t} \quad          t \geqslant 20$ Myr

Note the sign change from the paper because we're using a -z down sign convention. 

In [ ]:
# Build the GDH1 model
GDH1_t=np.arange(0,55,1); # age in Myr
GDH1_bath=np.zeros(len(GDH1_t)) # initialize size of depth array
GDH1_bath[GDH1_t<20]=...; # GDH1 model for bathymetry for young ages (<20 Myr)
GDH1_bath[GDH1_t>=20]=...; # GDH1 model for bathymetry for old ages (>20 Myr)


# plot the GDH1 model


##  Part 2: Bathymetry and Age, Data

We'll look at bathymetry data from several surveys around the pacific, and ages from our global age model.

In [ ]:
# Load the bathymetry data
pacific_data=pd.read_table('nbp9707.m77t')
pacific_data=pacific_data[~np.isnan(pacific_data.CORR_DEPTH)]
pacific_data = pacific_data[(pacific_data.LON>=-126.0) & (pacific_data.LON<=-95.0)]
pac_lat=pacific_data.LAT;
pac_lon=pacific_data.LON;
pac_depth=pacific_data.CORR_DEPTH;
pac_bath=-1*pac_depth;

south_pacific_data1=pd.read_table('elt42.m77t')
south_pacific_data1=south_pacific_data1[~np.isnan(south_pacific_data1.CORR_DEPTH)]
south_pacific_data1 = south_pacific_data1[(south_pacific_data1.LON>=-180.0) & (south_pacific_data1.LON<=-150.0)]
south_pac_lat1=south_pacific_data1.LAT;
south_pac_lon1=south_pacific_data1.LON;
south_pac_depth1=south_pacific_data1.CORR_DEPTH;
south_pac_bath1=-1*south_pac_depth1;

south_pacific_data2=pd.read_table('nbp95-7.m77t')
south_pacific_data2=south_pacific_data2[~np.isnan(south_pacific_data2.CORR_DEPTH)]
south_pacific_data2 = south_pacific_data2[(south_pacific_data2.LON>=-160.0) & (south_pacific_data2.LON<=-130.0)]
south_pac_lat2=south_pacific_data2.LAT;
south_pac_lon2=south_pacific_data2.LON;
south_pac_depth2=south_pacific_data2.CORR_DEPTH;
south_pac_bath2=-1*south_pac_depth2;




Open a track line from the Chile Ridge from file 'nbp97-4a.m77t'. Remove rows where depth is a nan. Select out rows where longitude is between -105 and -88 degrees. Declare variables for the latitude, longitude, and bathymetry.

In [ ]:
# Load the age data
age_data = pd.read_csv('age.csv')

age_longitude = np.asarray(age_data.Longitude)
age_latitude = np.asarray(age_data.Latitude)
age = np.asarray(age_data.Age_Myr)

A reminder about what the age data looks like:

<img src="Figures/age_map.png" >

In [ ]:
#subgrids of age_grid for 2d interpolation
pac_subgrid_lon = age_longitude[(age_longitude>-126) & (age_longitude<-95) & (age_latitude<(-0.1935*age_longitude-69.5)) & (age_latitude>(-0.1935*age_longitude-71.5))]
pac_subgrid_lat = age_latitude[(age_longitude>-126) & (age_longitude<-95) & (age_latitude<(-0.1935*age_longitude-69.5)) & (age_latitude>(-0.1935*age_longitude-71.5))]
pac_subgrid_age = age[(age_longitude>-126) & (age_longitude<-95) & (age_latitude<(-0.1935*age_longitude-69.5)) & (age_latitude>(-0.1935*age_longitude-71.5))]

south_pac_subgrid_lon = age_longitude[(age_longitude>-180) & (age_longitude<-150) & (age_latitude<(-0.26*age_longitude-104)) & (age_latitude>(-0.26*age_longitude-106))]
south_pac_subgrid_lat = age_latitude[(age_longitude>-180) & (age_longitude<-150) & (age_latitude<(-0.26*age_longitude-104)) & (age_latitude>(-0.26*age_longitude-106))]
south_pac_subgrid_age = age[(age_longitude>-180) & (age_longitude<-150) & (age_latitude<(-0.26*age_longitude-104)) & (age_latitude>(-0.26*age_longitude-106))]

south_pac_subgrid_lon2 = age_longitude[(age_longitude>-160) & (age_longitude<-130) & (age_latitude<(-0.1628*age_longitude-84)) & (age_latitude>(-0.1628*age_longitude-87))]
south_pac_subgrid_lat2 = age_latitude[(age_longitude>-160) & (age_longitude<-130) & (age_latitude<(-0.1628*age_longitude-84)) & (age_latitude>(-0.1628*age_longitude-87))]
south_pac_subgrid_age2 = age[(age_longitude>-160) & (age_longitude<-130) & (age_latitude<(-0.1628*age_longitude-84)) & (age_latitude>(-0.1628*age_longitude-87))]



For the Chile Ridge build a subgrid that pulls values from `age_longitude`, `age_latitude`, and `age` that have longitudes between -105 and -88 degrees and latitudes between -37 and -38 degrees.

In [ ]:
#Make a Map of one subgrid

#Set Corners of Map
lat0=-45
lat1=-53
lon0=-127
lon1=-94
tickstep=2.0 #for axes
latticks=np.arange(lat0,lat1+tickstep,tickstep)
lonticks=np.arange(lon0,lon1+tickstep,tickstep)

plt.figure(1,(15,15))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon0, lon1, lat0, lat1], crs=ccrs.PlateCarree())
ax.set_aspect('equal')
ax.set_xticks(lonticks)
ax.set_yticks(latticks, crs=ccrs.PlateCarree())
ax.set(xlabel='Longitude', ylabel='Latitude', title='East Pacific Rise Subgrid')


x=...
y=...
z=...
c = plt.cm.jet_r(z/max(age))

plt.scatter(x, y, facecolors=c, marker='.') # ages from age grid
plt.scatter(x[z<0.25], y[z<0.25], facecolors='white', marker='.') #highlight ridge
plt.plot(...,...,'.k') # track line where bathymetry data was collected


plt.show()

We want to interpolate the ages for the same locations as the bathymetry measurement. The `scipy` function `interpolate.Rbf` is really nice for doing this. You give `interpolate.Rbf` the x,y,z values of your function surface - in our case longitude, latitude, and age - to setup your interpolation function. Then you use that interpolation function to pickout the age values from the bathymetry measurements lon,lat locations. The new interoplated locations can have smaller spacing than the original scatter data (ages in our case).

In [ ]:
interp_pac = interpolate.Rbf(pac_subgrid_lon, pac_subgrid_lat, pac_subgrid_age, kind='multiquadratic') # interpolate subgridded ages
pac_age = interp_pac(pac_lon, pac_lat) # lon and lat values from bathymetry tracks, returns the age at that location

interp_south_pac1 = interpolate.Rbf(south_pac_subgrid_lon, south_pac_subgrid_lat, south_pac_subgrid_age, kind='multiquadratic') # interpolate subgridded ages
south_pac_age1 = interp_south_pac1(south_pac_lon1, south_pac_lat1) # lon and lat values from bathymetry tracks, returns the age at that location

interp_south_pac2 = interpolate.Rbf(south_pac_subgrid_lon2, south_pac_subgrid_lat2, south_pac_subgrid_age2, kind='multiquadratic') # interpolate subgridded ages
south_pac_age2 = interp_south_pac2(south_pac_lon2, south_pac_lat2) # lon and lat values from bathymetry tracks, returns the age at that location



Build an interpolation function using `interpolate.Rbf` and the Chile Ridge subgrid, then apply it to the locations of the bathymetry measurements over that ridge.

Plot our data: age and bathymetry.

In [ ]:
plt.figure(1,(20,10))
plt.plot(chile_age,chile_bath,'.',color='firebrick');
plt.plot(south_pac_age1,south_pac_bath1,'.',color='lightcoral');
plt.plot(south_pac_age2,south_pac_bath2,'.',color='orange');
plt.plot(pac_age,pac_bath,'.',color='red');
plt.legend(['Chile','EPR','PAR-1','PAR-2'],fontsize=16)
plt.xlabel('Age, Myr',fontsize=16);
plt.ylabel('Bathymentry, m',fontsize=16);

##  Part 3: Bathymetry and Age, Model

We'll start from the GDH1 model of Stein and Stein, 1992.

In [ ]:
# plot our data and the GHD1 model
plt.figure(1,(20,10))
plt.plot(chile_age,chile_bath,'.',color='firebrick');
plt.plot(south_pac_age1,south_pac_bath1,'.',color='lightcoral');
plt.plot(south_pac_age2,south_pac_bath2,'.',color='orange');
plt.plot(pac_age,pac_bath,'.',color='red');
plt.plot(GDH1_t,GDH1_bath,'-k',Linewidth=3);
plt.legend(['Chile','EPR','PAR-1','PAR-2','GDH1'],fontsize=16)
plt.xlabel('Age, Myr',fontsize=16);
plt.ylabel('Bathymentry, m',fontsize=16);

Time to make our own model of bathymetry as a function of age! We'll use `polyfit` and `polyval` along with our knowledge about the predicted relationship.

In [ ]:
#use hstack to combine all the data tracks
all_age = np.hstack((chile_age, south_pac_age1, south_pac_age2, pac_age));
all_bath = np.hstack((chile_bath, south_pac_bath1, south_pac_bath2, pac_bath));

# fit the young data (age<20Myr)
young_age = all_age[all_age<20];
young_bath = all_bath[all_age<20];
soln1=np.polyfit(...,...,...)
Model_young_bath = np.polyval(...,...)


# fit the old data (age>=20Myr)
old_age = all_age[all_age>=20];
old_bath = all_bath[all_age>=20];
soln2=np.polyfit(...,...,...)
Model_old_bath = np.polyval(...,...)


Print our model parameters

In [ ]:
print ('Bathymetry of ridge: %4.1f, Young Coeff: %4.1f'% (soln1[1],soln1[0]))
print ('dr+ds : %4.1f, old Coeff: %4.1f'% (soln2[1],soln2[0]))

How does our model parameters compare to those of GDH1 model?

$b(t) = -2600 - 365 \sqrt{t} \quad  \quad        t < 20$ Myr

$b(t) = -5651 + 2473 e^{-0.0278t} \quad          t \geqslant 20$ Myr

_Write your answer here._

Before we plot our model, how do you expect it differ from GDH1? Are the ridge depth and asymptotic depth deeper or shallower? 

In [ ]:
# plot our data, the GHD1 model, and our model
plt.figure(1,(20,10))


What are some reasons for the differences between our model and GDH1 (what did they do differently)?

_Write your answer here._

If we assume all the same model values for densities, coeff. of thermal expansion, and thermal diffusivity that Stein and Stein (1992) used, what mantle temperature does out model predict?  

$C_{young} \propto Tm$ 

$\frac{Tm_{class}}{C_{class}} = \frac{Tm_{GDH1}}{C_{GDH1}} $ 

$Tm_{class} = \frac{Tm_{GDH1}\times C_{class}}{C_{GDH1}}$

In [ ]:
GDH1_Tm = 1450;
GDH1_coeff = -365;
Tm_class=...
print ('Mantle Temp : %4.0f degrees Celsius'% (Tm_class))

Does this agree with the mantle temperature predicted by Stein and Stein (1992) within their stated uncertainty?

_Write your answer here._